## 1.1 BCI dataset

In [2]:
%python
import pandas as pd
bci_pdf = pd.read_csv('/dbfs/mnt/group-ma707/data/5tc_plus_ind_vars.csv') \
            .rename(columns={'P3A~IV':'P3A_IV'}) \
            .assign(date=lambda pdf: pd.to_datetime(pdf.Date)) \
            .drop('Date', axis=1) \
            .sort_index(ascending=True)
bci_pdf.columns = bci_pdf.columns.str.lower()
bci_pdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1602 entries, 0 to 1601
Data columns (total 26 columns):
bci 1602 non-null int64
c5 1602 non-null float64
c7 1602 non-null float64
p1a_03 1602 non-null int64
p2a_03 1602 non-null int64
p4_03 1602 non-null int64
p3a_iv 1602 non-null float64
shfe_al3 1602 non-null float64
rici 1602 non-null float64
ice_kc3 1602 non-null float64
cme_sm3 1602 non-null float64
cme_lc2 1602 non-null float64
opec_orb 1602 non-null float64
shfe_cu3 1602 non-null float64
cme_ln1 1602 non-null float64
cme_fc3 1602 non-null float64
p3a_03 1602 non-null int64
shfe_rb3 1602 non-null int64
cme_s2 1602 non-null float64
ice_sb3 1602 non-null float64
cme_ln3 1602 non-null float64
cme_ln2 1602 non-null float64
ice_tib3 1602 non-null float64
ice_tib4 1602 non-null float64
bci_5tc 1602 non-null int64
date 1602 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(18), int64(7)
memory usage: 325.5 KB

## 1.2 Coal dataset

In [4]:
%python
import numpy as np
import pandas as pd
coal_pdf = \
pd.read_csv('/dbfs/mnt/group-ma707/data/mining_com_coal.csv', 
            encoding='ISO-8859-1'
           ) \
  .dropna(axis=0, subset=['tags','content','title']) \
  .assign(date   =lambda pdf: pd.to_datetime(pd.to_datetime(pdf.date).dt.date)
         ) \
  .loc[:,['date','tags','title','content']] \
  .sort_values('date', ascending=True)
coal_pdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4453 entries, 4459 to 0
Data columns (total 4 columns):
date 4453 non-null datetime64[ns]
tags 4453 non-null object
title 4453 non-null object
content 4453 non-null object
dtypes: datetime64[ns](1), object(3)
memory usage: 173.9+ KB

In [5]:
%python 
dup_date_ser = \
coal_pdf['date'] \
  .value_counts() \
  .sort_index(ascending=True) \
  .loc[lambda ser:ser==2] 
dup_date_ser \
  .tail()

Out[ 3 ]: 
2018-12-17 2
2018-12-26 2
2018-12-27 2
2019-01-13 2
2019-01-15 2
Name: date, dtype: int64

In [6]:
dup_date = dup_date_ser.index.date[-1]
dup_date

Out[ 4 ]: datetime.date(2019, 1, 15)

In [7]:
%python coal_pdf.set_index('date').loc[dup_date].tags

Out[ 5 ]: 
date
2019-01-15 africa coal south-africa
2019-01-15 britain china coal cobalt copper europe graphi...
Name: tags, dtype: object

In [8]:
%python 
coal_pdf.groupby(by='date').sum().loc[dup_date].tags

Out[ 6 ]: 'africa coal south-africabritain china coal cobalt copper europe graphite lithium nickel u-s-a'

Problem above. Fix below.

Put space at the end of each value in the `tags`, `title` and `content` columns. See code cell below.

In [10]:
%python
import numpy as np
import pandas as pd
coal_pdf = \
pd.read_csv('/dbfs/mnt/group-ma707/data/mining_com_coal.csv', 
            encoding='ISO-8859-1'
           ) \
  .dropna(axis=0, subset=['tags','content','title']) \
  .assign(date   =lambda pdf: pd.to_datetime(pd.to_datetime(pdf.date).dt.date),
          tags   =lambda pdf: pd.Series(data=[tags   +' ' for tags    in list(pdf.tags)]),
          content=lambda pdf: pd.Series(data=[content+' ' for content in list(pdf.content)]),
          title  =lambda pdf: pd.Series(data=[title  +' ' for title   in list(pdf.title)])
         ) \
  .loc[:,['date','tags','title','content']] \
  .sort_values('date', ascending=True)
coal_pdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4453 entries, 4459 to 0
Data columns (total 4 columns):
date 4453 non-null datetime64[ns]
tags 4446 non-null object
title 4446 non-null object
content 4446 non-null object
dtypes: datetime64[ns](1), object(3)
memory usage: 173.9+ KB

In [11]:
%python coal_pdf.set_index('date').loc[dup_date].tags

Out[ 8 ]: 
date
2019-01-15 africa coal south-africa 
2019-01-15 britain china coal cobalt copper europe graphi...
Name: tags, dtype: object

In [12]:
%python 
coal_pdf.groupby(by='date').sum().loc[dup_date].tags

Out[ 9 ]: 'africa coal south-africa britain china coal cobalt copper europe graphite lithium nickel u-s-a '

In [13]:
%python
import numpy as np
import pandas as pd
coal_pdf = \
pd.read_csv('/dbfs/mnt/group-ma707/data/mining_com_coal.csv', 
            encoding='ISO-8859-1'
           ) \
  .dropna(axis=0, subset=['tags','content','title']) \
  .assign(date   =lambda pdf: pd.to_datetime(pd.to_datetime(pdf.date).dt.date),
          tags   =lambda pdf: pd.Series(data=[tags   +' ' for tags    in list(pdf.tags)]),
          content=lambda pdf: pd.Series(data=[content+' ' for content in list(pdf.content)]),
          title  =lambda pdf: pd.Series(data=[title  +' ' for title   in list(pdf.title)])
         ) \
  .loc[:,['date','tags','title','content']] \
  .groupby(by='date') \
  .sum() \
  .reset_index() \
  .sort_values('date', ascending=True)
coal_pdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2231 entries, 0 to 2230
Data columns (total 4 columns):
date 2231 non-null datetime64[ns]
tags 2231 non-null object
title 2231 non-null object
content 2231 non-null object
dtypes: datetime64[ns](1), object(3)
memory usage: 87.1+ KB

In [14]:
%python 
coal_pdf['date'] \
  .value_counts() \
  .sort_index(ascending=True) \
  .value_counts()

Out[ 11 ]: 
1 2231
Name: date, dtype: int64

In [15]:
coal_pdf.date.dt.weekday.value_counts()

Out[ 12 ]: 
0 394
3 393
1 383
2 378
4 357
6 210
5 116
Name: date, dtype: int64

In [16]:
%python 
coal_pdf.set_index('date').sort_index(ascending=True).head(10) # debug

Out[ 13 ]: 
 tags ... content
date ... 
2008-04-01 0 ... 0
2008-04-07 0 ... 0
2008-04-10 0 ... 0
2008-04-11 0 ... 0
2008-04-15 0 ... 0
2008-04-16 0 ... 0
2008-04-17 0 ... 0
2008-04-18 climate coal coal-miners community development... ... National Mining Association (NMA) President an...
2008-04-21 coal development dust emissions liner benefici... ... Sasol, the worldâs leading producer of synth...
2008-04-25 coal mine-safety-and-health-administration msh... ... The Mine Safety and Health Administration (MSH...

[10 rows x 3 columns]

In [17]:
%python 
coal_pdf.set_index('date').sort_index(ascending=True).tags[-1] #.tail(10) # debug

Out[ 14 ]: 'britain china coal cobalt copper europe graphite lithium nickel u-s-a africa coal south-africa '

In [18]:
%python 
coal_pdf.set_index('date').resample('D').pad().sort_index(ascending=True).tail(20)

Out[ 15 ]: 
 tags ... content
date ... 
2018-12-27 africa asia australia canada china coal copper... ... (This article first appeared May 2017)Nealite ...
2018-12-28 coal murray-energy-corp u-s-a ... The Trump administration is proposing that lim...
2018-12-29 coal murray-energy-corp u-s-a ... The Trump administration is proposing that lim...
2018-12-30 coal murray-energy-corp u-s-a ... The Trump administration is proposing that lim...
2018-12-31 coal murray-energy-corp u-s-a ... The Trump administration is proposing that lim...
2019-01-01 coal murray-energy-corp u-s-a ... The Trump administration is proposing that lim...
2019-01-02 coal murray-energy-corp u-s-a ... The Trump administration is proposing that lim...
2019-01-03 coal murray-energy-corp u-s-a ... The Trump administration is proposing that lim...
2019-01-04 asia coal india ... Indian rescuers trying to reach a group of min...
2019-01-05 asia coal india ... Indian rescuers trying to reach a group of min...
2019-01-06 asia coal india ... Indian rescuers trying to reach a group of min...
2019-01-07 asia coal india ... Indian rescuers trying to reach a group of min...
2019-01-08 asia coal india ... Indian rescuers trying to reach a group of min...
2019-01-09 asia coal india ... Indian rescuers trying to reach a group of min...
2019-01-10 asia coal india ... Indian rescuers trying to reach a group of min...
2019-01-11 africa arabesque asia coal copper esg-data eth... ... Both companies and the fund managers that inve...
2019-01-12 africa arabesque asia coal copper esg-data eth... ... Both companies and the fund managers that inve...
2019-01-13 coal europe germany baiji-mining-co china coal ... Angela Merkel this week will seek to bridge di...
2019-01-14 china coal iron-ore steel ... * China's 2018 iron ore imports fall to 1.064 ...
2019-01-15 britain china coal cobalt copper europe graphi... ... Electric cars are better for the environment t...

[20 rows x 3 columns]

In [19]:
%python 
coal_pdf.set_index('date').resample('D').pad().reset_index().date.dt.weekday.value_counts()

Out[ 16 ]: 
1 564
5 563
3 563
6 563
4 563
2 563
0 563
Name: date, dtype: int64

In [20]:
%python
import numpy as np
import pandas as pd
coal_pdf = \
pd.read_csv('/dbfs/mnt/group-ma707/data/mining_com_coal.csv', 
            encoding='ISO-8859-1'
           ) \
  .dropna(axis=0, subset=['tags','content','title']) \
  .assign(date   =lambda pdf: pd.to_datetime(pd.to_datetime(pdf.date).dt.date),
          tags   =lambda pdf: pd.Series(data=[tags   +' ' for tags    in list(pdf.tags)]),
          content=lambda pdf: pd.Series(data=[content+' ' for content in list(pdf.content)]),
          title  =lambda pdf: pd.Series(data=[title  +' ' for title   in list(pdf.title)])
         ) \
  .loc[:,['date','tags','title','content']] \
  .groupby(by='date') \
  .sum() \
  .resample('D') \
  .pad() \
  .reset_index() \
  .sort_values('date', ascending=True) \
  .add_suffix('_coal') \
  .rename(columns={"date_coal": "date"})
coal_pdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3942 entries, 0 to 3941
Data columns (total 4 columns):
date 3942 non-null datetime64[ns]
tags_coal 3942 non-null object
title_coal 3942 non-null object
content_coal 3942 non-null object
dtypes: datetime64[ns](1), object(3)
memory usage: 154.0+ KB

## 1.3 Iron ore dataset

In [22]:
%python
import numpy as np
import pandas as pd
ore_pdf = \
pd.read_csv('/dbfs/mnt/group-ma707/data/mining_com_iron_ore.csv', 
            encoding='ISO-8859-1'
           ) \
  .loc[:,['date','tags','title','content']] \
  .fillna({'tags'   :'',
           'content':'',
           'title'  :''
          }) \
  .assign(date = lambda pdf: pd.to_datetime(pd.to_datetime(pdf.date,utc=True).dt.normalize().dt.date)) \
  .groupby(by='date') \
  .agg({'tags'   : lambda ser: ' '.join(ser),
        'content': lambda ser: ' '.join(ser),
        'title'  : lambda ser: ' '.join(ser)}) \
  .resample('D') \
  .pad() \
  .reset_index() \
  .add_suffix('_ore') \
  .rename(columns={"date_ore": "date"})
ore_pdf.info(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3928 entries, 0 to 3927
Data columns (total 4 columns):
date 3928 non-null datetime64[ns]
content_ore 3928 non-null object
tags_ore 3928 non-null object
title_ore 3928 non-null object
dtypes: datetime64[ns](1), object(3)
memory usage: 122.8+ KB

In [23]:
coal_pdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3942 entries, 0 to 3941
Data columns (total 4 columns):
date 3942 non-null datetime64[ns]
tags_coal 3942 non-null object
title_coal 3942 non-null object
content_coal 3942 non-null object
dtypes: datetime64[ns](1), object(3)
memory usage: 154.0+ KB

In [24]:
bci_pdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1602 entries, 0 to 1601
Data columns (total 26 columns):
bci 1602 non-null int64
c5 1602 non-null float64
c7 1602 non-null float64
p1a_03 1602 non-null int64
p2a_03 1602 non-null int64
p4_03 1602 non-null int64
p3a_iv 1602 non-null float64
shfe_al3 1602 non-null float64
rici 1602 non-null float64
ice_kc3 1602 non-null float64
cme_sm3 1602 non-null float64
cme_lc2 1602 non-null float64
opec_orb 1602 non-null float64
shfe_cu3 1602 non-null float64
cme_ln1 1602 non-null float64
cme_fc3 1602 non-null float64
p3a_03 1602 non-null int64
shfe_rb3 1602 non-null int64
cme_s2 1602 non-null float64
ice_sb3 1602 non-null float64
cme_ln3 1602 non-null float64
cme_ln2 1602 non-null float64
ice_tib3 1602 non-null float64
ice_tib4 1602 non-null float64
bci_5tc 1602 non-null int64
date 1602 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(18), int64(7)
memory usage: 325.5 KB

In [25]:
ore_pdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3928 entries, 0 to 3927
Data columns (total 4 columns):
date 3928 non-null datetime64[ns]
content_ore 3928 non-null object
tags_ore 3928 non-null object
title_ore 3928 non-null object
dtypes: datetime64[ns](1), object(3)
memory usage: 122.8+ KB

## 1.4 Initial dataset (TBD)

The "initial dataset" is created by merging the BCI and the two mining datasets.
Below the coal and bci datasets are merged to create the `bci_coal_pdf` datasets/dataframe.

In [27]:
%python
import pandas as pd
bci_coal_pdf = \
pd.concat(objs=[ bci_pdf.set_index('date'), 
                coal_pdf.set_index('date')], 
          join='inner',
          axis=1
         ) \
  .reset_index()
bci_coal_pdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1597 entries, 0 to 1596
Data columns (total 29 columns):
date 1597 non-null datetime64[ns]
bci 1597 non-null int64
c5 1597 non-null float64
c7 1597 non-null float64
p1a_03 1597 non-null int64
p2a_03 1597 non-null int64
p4_03 1597 non-null int64
p3a_iv 1597 non-null float64
shfe_al3 1597 non-null float64
rici 1597 non-null float64
ice_kc3 1597 non-null float64
cme_sm3 1597 non-null float64
cme_lc2 1597 non-null float64
opec_orb 1597 non-null float64
shfe_cu3 1597 non-null float64
cme_ln1 1597 non-null float64
cme_fc3 1597 non-null float64
p3a_03 1597 non-null int64
shfe_rb3 1597 non-null int64
cme_s2 1597 non-null float64
ice_sb3 1597 non-null float64
cme_ln3 1597 non-null float64
cme_ln2 1597 non-null float64
ice_tib3 1597 non-null float64
ice_tib4 1597 non-null float64
bci_5tc 1597 non-null int64
tags_coal 1597 non-null object
title_coal 1597 non-null object
content_coal 1597 non-null object
dtypes: datetime64[ns](1), float64(18), int64(7), object(3)
memory usage: 361.9+ KB

## 1.5 Initial dataset (TBD)

The "initial dataset" is created by merging the BCI and the two mining datasets.
Below the ironore and bci datasets are merged to create the `bci_ironore_pdf` datasets/dataframe.

In [29]:
%python
import pandas as pd
bci_ironore_pdf = \
pd.concat(objs=[bci_pdf.set_index('date'), 
                ore_pdf.set_index('date')], 
          join='inner',
          axis=1
         ) \
  .reset_index()
bci_ironore_pdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1598 entries, 0 to 1597
Data columns (total 29 columns):
date 1598 non-null datetime64[ns]
bci 1598 non-null int64
c5 1598 non-null float64
c7 1598 non-null float64
p1a_03 1598 non-null int64
p2a_03 1598 non-null int64
p4_03 1598 non-null int64
p3a_iv 1598 non-null float64
shfe_al3 1598 non-null float64
rici 1598 non-null float64
ice_kc3 1598 non-null float64
cme_sm3 1598 non-null float64
cme_lc2 1598 non-null float64
opec_orb 1598 non-null float64
shfe_cu3 1598 non-null float64
cme_ln1 1598 non-null float64
cme_fc3 1598 non-null float64
p3a_03 1598 non-null int64
shfe_rb3 1598 non-null int64
cme_s2 1598 non-null float64
ice_sb3 1598 non-null float64
cme_ln3 1598 non-null float64
cme_ln2 1598 non-null float64
ice_tib3 1598 non-null float64
ice_tib4 1598 non-null float64
bci_5tc 1598 non-null int64
content_ore 1598 non-null object
tags_ore 1598 non-null object
title_ore 1598 non-null object
dtypes: datetime64[ns](1), float64(18), int64(7), object(3)
memory usage: 362.1+ KB

## 1.6 Initial dataset (TBD)

The "initial dataset" is created by merging the BCI and the two mining datasets.
Below the bci_pdf, coal_pdf and ore_pdf datasets are merged to create the `bci_all_dual_pdf` datasets/dataframe.

In [31]:
bci_dual_pdf = \
pd.concat(objs=[ bci_pdf.set_index('date'), 
                 ore_pdf.set_index('date'),
                 coal_pdf.set_index('date')], 
          join='inner',
          axis=1
         ) \
  .reset_index()
bci_dual_pdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1597 entries, 0 to 1596
Data columns (total 32 columns):
date 1597 non-null datetime64[ns]
bci 1597 non-null int64
c5 1597 non-null float64
c7 1597 non-null float64
p1a_03 1597 non-null int64
p2a_03 1597 non-null int64
p4_03 1597 non-null int64
p3a_iv 1597 non-null float64
shfe_al3 1597 non-null float64
rici 1597 non-null float64
ice_kc3 1597 non-null float64
cme_sm3 1597 non-null float64
cme_lc2 1597 non-null float64
opec_orb 1597 non-null float64
shfe_cu3 1597 non-null float64
cme_ln1 1597 non-null float64
cme_fc3 1597 non-null float64
p3a_03 1597 non-null int64
shfe_rb3 1597 non-null int64
cme_s2 1597 non-null float64
ice_sb3 1597 non-null float64
cme_ln3 1597 non-null float64
cme_ln2 1597 non-null float64
ice_tib3 1597 non-null float64
ice_tib4 1597 non-null float64
bci_5tc 1597 non-null int64
content_ore 1597 non-null object
tags_ore 1597 non-null object
title_ore 1597 non-null object
tags_coal 1597 non-null object
title_coal 1597 non-null object
content_coal 1597 non-null object
dtypes: datetime64[ns](1), float64(18), int64(7), object(6)
memory usage: 399.3+ KB